In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit


# Data Preparation

(See notes on data preparation in xgboost.ipynb)

In [36]:
df = pd.read_csv('all_hourly_data.csv')

In [37]:
print(df.shape)
df = df.drop(['date'], axis=1) #we're adding dummies later for this

(63689, 25)


Let's generate lags for

- The previous 24 hours (i.e., lags 1 to 24),
- The same hour 2 and 3 days ago (lags 48 and 72),
- The same hour 1 week ago (lag 168),

Let's also generate hourly, 2-3 days, and weekly dummies.

In [38]:
#handle the time column
df['interval_start_local'] = pd.to_datetime(df['interval_start_local'], utc=True)
df = df.set_index('interval_start_local')

#lagging 
cols_to_lag = df.columns
lags = list(range(1, 25)) + [48, 72, 168]

lagged_frames = []
for lag in lags:
    lagged = df[cols_to_lag].shift(lag)
    lagged.columns = [f"{col}_lag{lag}" for col in cols_to_lag]
    lagged_frames.append(lagged)

lagged_df = pd.concat(lagged_frames, axis=1)
df_with_lags = pd.concat([df, lagged_df], axis=1).reset_index()
df_with_lags = df_with_lags.copy()

#Lose the first 168 observations because our largest lag is 1 week long
df_with_lags = df_with_lags.iloc[168:].reset_index(drop=True)


In [39]:
df_with_lags['hour'] = pd.to_datetime(df_with_lags['interval_start_local']).dt.hour
df_with_lags['dayofweek'] = pd.to_datetime(df_with_lags['interval_start_local']).dt.dayofweek

#one-hot encode dummies
hour_dummies = pd.get_dummies(df_with_lags['hour'], prefix='hour')
dow_dummies = pd.get_dummies(df_with_lags['dayofweek'], prefix='dow')
hour_dummies = hour_dummies.astype(int)
dow_dummies = dow_dummies.astype(int)

#merge
df = pd.concat([df_with_lags, hour_dummies, dow_dummies], axis=1)
df = df.drop(['hour', 'dayofweek'], axis=1)

print(df.head())

       interval_start_local  north_x_load  south_load  west_x_load  \
0 2018-01-08 06:00:00+00:00      11292.28     7720.14      3626.51   
1 2018-01-08 07:00:00+00:00      11134.52     7540.80      3615.65   
2 2018-01-08 08:00:00+00:00      11183.56     7529.38      3624.71   
3 2018-01-08 09:00:00+00:00      11497.37     7687.16      3678.35   
4 2018-01-08 10:00:00+00:00      12203.96     8053.77      3761.58   

   houston_load  total_load  lmp_HB_BUSAVG  coal_and_lignite      hydro  \
0       8181.37    30820.31      18.535000       9942.516942  39.045263   
1       7980.93    30271.90      18.107500      10059.864300  15.672631   
2       7866.77    30204.42      18.019167       9912.115128  14.312730   
3       7874.74    30737.62      18.026667       9731.137220  14.721392   
4       8095.04    32114.36      18.192500       9950.836098  15.266061   

       nuclear  ...  hour_21  hour_22  hour_23  dow_0  dow_1  dow_2  dow_3  \
0  5123.325668  ...        0        0        0    

In [40]:
#'week of the year' dummies

df['weekofyear'] = pd.to_datetime(df['interval_start_local']).dt.isocalendar().week

weekofyear_dummies = pd.get_dummies(df['weekofyear'], prefix='weekofyear', drop_first=False)
weekofyear_dummies = weekofyear_dummies.astype(int)

df = pd.concat([df, weekofyear_dummies], axis=1)
df = df.drop(['weekofyear'], axis=1)

# Preview result
print(df.head())

       interval_start_local  north_x_load  south_load  west_x_load  \
0 2018-01-08 06:00:00+00:00      11292.28     7720.14      3626.51   
1 2018-01-08 07:00:00+00:00      11134.52     7540.80      3615.65   
2 2018-01-08 08:00:00+00:00      11183.56     7529.38      3624.71   
3 2018-01-08 09:00:00+00:00      11497.37     7687.16      3678.35   
4 2018-01-08 10:00:00+00:00      12203.96     8053.77      3761.58   

   houston_load  total_load  lmp_HB_BUSAVG  coal_and_lignite      hydro  \
0       8181.37    30820.31      18.535000       9942.516942  39.045263   
1       7980.93    30271.90      18.107500      10059.864300  15.672631   
2       7866.77    30204.42      18.019167       9912.115128  14.312730   
3       7874.74    30737.62      18.026667       9731.137220  14.721392   
4       8095.04    32114.36      18.192500       9950.836098  15.266061   

       nuclear  ...  weekofyear_44  weekofyear_45  weekofyear_46  \
0  5123.325668  ...              0              0           

In [41]:
#drop one dummy for each category to prevent multicollinearity
df = df.drop(['weekofyear_53', 'dow_6', 'hour_23'], axis=1)

In [42]:
#define date ranges for each set
train_start = pd.Timestamp('2018-01-08 06:00:00+0000', tz='UTC')
train_end = pd.Timestamp('2022-12-31 23:59:59+0000', tz='UTC')

val_start = pd.Timestamp('2022-01-01 00:00:00+0000', tz='UTC')
val_end = pd.Timestamp('2023-06-01 00:00:00+0000', tz='UTC')

test_start = pd.Timestamp('2023-06-01 00:00:00+0000', tz='UTC')
test_end = pd.Timestamp('2025-04-13 04:00:00+0000', tz='UTC')

#filter
train_data = df[(df['interval_start_local'] >= train_start) & (df['interval_start_local'] <= train_end)]
val_data = df[(df['interval_start_local'] >= val_start) & (df['interval_start_local'] <= val_end)]
test_data = df[(df['interval_start_local'] >= test_start) & (df['interval_start_local'] <= test_end)]

#check dims
print(f"Train set size: {train_data.shape[0]}")
print(f"Validation set size: {val_data.shape[0]}")
print(f"Test set size: {test_data.shape[0]}")

Train set size: 43650
Validation set size: 12385
Test set size: 16247


Data for regression trees are not scaled since tree-based models work by splitting the data based on feature values, not by calculating distances or assuming a particular scale. 

In [43]:
#preview the data
print("Training data preview:\n", train_data.head())
print("Validation data preview:\n", val_data.head())
print("Test data preview:\n", test_data.head())

Training data preview:
        interval_start_local  north_x_load  south_load  west_x_load  \
0 2018-01-08 06:00:00+00:00      11292.28     7720.14      3626.51   
1 2018-01-08 07:00:00+00:00      11134.52     7540.80      3615.65   
2 2018-01-08 08:00:00+00:00      11183.56     7529.38      3624.71   
3 2018-01-08 09:00:00+00:00      11497.37     7687.16      3678.35   
4 2018-01-08 10:00:00+00:00      12203.96     8053.77      3761.58   

   houston_load  total_load  lmp_HB_BUSAVG  coal_and_lignite      hydro  \
0       8181.37    30820.31      18.535000       9942.516942  39.045263   
1       7980.93    30271.90      18.107500      10059.864300  15.672631   
2       7866.77    30204.42      18.019167       9912.115128  14.312730   
3       7874.74    30737.62      18.026667       9731.137220  14.721392   
4       8095.04    32114.36      18.192500       9950.836098  15.266061   

       nuclear  ...  weekofyear_43  weekofyear_44  weekofyear_45  \
0  5123.325668  ...              0  

# Tuning the model

Trying out the following:
- Binary regression decision tree
- Bagged regression tree
- Boosted regression tree

## Binary regression decision tree

In [46]:
print(train_data.columns.tolist())

['interval_start_local', 'north_x_load', 'south_load', 'west_x_load', 'houston_load', 'total_load', 'lmp_HB_BUSAVG', 'coal_and_lignite', 'hydro', 'nuclear', 'power_storage', 'solar', 'wind', 'natural_gas', 'other_gen', 'coast_temp', 'east_temp', 'far_west_temp', 'north_y_temp', 'north_central_temp', 'south_central_temp', 'southern_temp', 'west_y_temp', 'hour_lag1', 'north_x_load_lag1', 'south_load_lag1', 'west_x_load_lag1', 'houston_load_lag1', 'total_load_lag1', 'lmp_HB_BUSAVG_lag1', 'coal_and_lignite_lag1', 'hydro_lag1', 'nuclear_lag1', 'power_storage_lag1', 'solar_lag1', 'wind_lag1', 'natural_gas_lag1', 'other_gen_lag1', 'coast_temp_lag1', 'east_temp_lag1', 'far_west_temp_lag1', 'north_y_temp_lag1', 'north_central_temp_lag1', 'south_central_temp_lag1', 'southern_temp_lag1', 'west_y_temp_lag1', 'hour_lag2', 'north_x_load_lag2', 'south_load_lag2', 'west_x_load_lag2', 'houston_load_lag2', 'total_load_lag2', 'lmp_HB_BUSAVG_lag2', 'coal_and_lignite_lag2', 'hydro_lag2', 'nuclear_lag2', 'p

In [ ]:
tscv = TimeSeriesSplit(n_splits=10)

target_col = 'lmp_HB_BUSAVG'
# dropping interval_start_local because relevant time features like hour, dayofweek, and weekofyear are already included, datetime stamps themselves 
# are not meaningful for decision trees
X = train_data.drop(columns = ['interval_start_local', target_col])
y = train_data[target_col]

First, estimate the model complexity of unrestrictred trees using time-series cross validation. This defines a cap to max_leaf_nodes (number of nodes) to prevent overfitting.

In [ ]:
#list to store the number of splits for each fold in trianing data
num_splits_list = []


Average number of splits: 21797.90
Setting max_leaf_nodes = 10899


In [ ]:

#Splitting training data into smaller train and validation sets in each fold for hyperparameter tuning
for train_index, val_index in tscv.split(X):
    X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
    y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

    tree = DecisionTreeRegressor(random_state=42)
    tree.fit(X_tr, y_tr)

    tree_structure = tree.tree_
    num_splits = tree_structure.node_count - tree_structure.n_leaves
    num_splits_list.append(num_splits)

avg_splits = np.mean(num_splits_list)
print(f"Average number of splits: {avg_splits:.2f}")

max_leaf_nodes_limit = int((avg_splits / 2) + 1)
print(f"Setting max_leaf_nodes = {max_leaf_nodes_limit}")

Next, tuning is done to find out the best number of samples per leaf using grid search from 1 to 50.

In [ ]:
# Tune min_samples_leaf from 1 to 50
rmse_per_leaf_size = []

for min_leaf in range(1, 51):
    fold_rmse = []
    for train_index, val_index in tscv.split(X):
        X_tr, X_val = X.iloc[train_index], X.iloc[val_index]
        y_tr, y_val = y.iloc[train_index], y.iloc[val_index]

        tree = DecisionTreeRegressor(
            min_samples_leaf=min_leaf,
            max_leaf_nodes=max_leaf_nodes_limit,
            random_state=42
        )
        tree.fit(X_tr, y_tr)
        preds = tree.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_rmse.append(rmse)
    
    avg_rmse = np.mean(fold_rmse)
    rmse_per_leaf_size.append((min_leaf, avg_rmse))

In [ ]:
# Best hyperparameter
best_leaf, best_rmse = min(rmse_per_leaf_size, key=lambda x: x[1])
print(f"\nBest min_samples_leaf: {best_leaf} with average RMSE: {best_rmse:.4f}")

## Boosted regression tree with LightGBM

LightGBM uses leaf-wise tree growth (vs XGBoost's level-wise), which can lead to deeper trees and better accuracy, but risk for overfitting possible.

LightGBM splits leaves with largest loss reduction, regardless of depth, so it favors deep uneven trees (while XGBoost splits all leaves at the current depth before moving deeper).

LightGBM used in this paper: https://ieeexplore.ieee.org/document/10693001

# Comparing the different trees

In [ ]:
#Final Decision Tree model (best from CV)
tree_model = DecisionTreeRegressor(
    min_samples_leaf=best_leaf,
    max_leaf_nodes=max_leaf_nodes_limit,
    random_state=42
)
tree_model.fit(X_train, y_train)

#Final bagged model (best from OOB MAE tuning)
bagged_model = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    n_estimators=int(best_n_trees['n_trees']),  # based on OOB MAE tuning
    oob_score=True,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)
bagged_model.fit(X_train, y_train)

# Final Boosted Tree model (best from tuning)
boosted_model = GradientBoostingRegressor(
    learning_rate=best_lr,
    n_estimators=n_estimators,
    max_depth=int(np.log2(max_splits + 1)),
    loss='squared_error',
    random_state=42
)
boosted_model.fit(X_train, y_train)

#Predict on training data
tree_preds_train = tree_model.predict(X_train)
bagged_preds_train = bagged_model.predict(X_train)
boosted_preds_train = boosted_model.predict(X_train)

#Compare MAE
tree_mae = np.sqrt(mean_absolute_error(y_train, tree_preds_train))
bagged_mae = np.sqrt(mean_absolute_error(y_train, bagged_preds_train))
boosted_mae = np.sqrt(mean_absolute_error(y_train, boosted_preds_train))

#Compute RMSE
tree_rmse = np.sqrt(mean_squared_error(y_train, tree_preds_train))
bagged_rmse = np.sqrt(mean_squared_error(y_train, bagged_preds_train))
boosted_rmse = np.sqrt(mean_squared_error(y_train, boosted_preds_train))

comparison_df = pd.DataFrame({
    "Model": ["Decision Tree", "Bagged Trees", "Boosted Trees"],
    "Train RMSE": [tree_rmse, bagged_rmse, boosted_rmse]
})

print(comparison_df)

           Model  Train RMSE
0  Decision Tree  116.491969
1   Bagged Trees   41.817129
2  Boosted Trees   39.944288
